In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.porter import PorterStemmer

In [2]:
# HELPER FUNCTIONS......

def encoding_discourse_type(x):
    if x=="Lead":
        return 0
    if x=="Position":
        return 1
    if x=="Evidence":
        return 2
    if x=="Claim":
        return 3
    if x=="Concluding Statement":
        return 4
    if x=="Counterclaim":
        return 5
    if x=='Rebuttal':
        return 6

In [3]:
df=pd.read_csv("train.csv")
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [4]:
len(df["id"].unique())

15594

In [5]:
df.groupby(["id"]).head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...
144284,4C471936CD75,1.618025e+12,0.0,570.0,"In ancient times, and also still today in some...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
144285,4C471936CD75,1.618025e+12,731.0,844.0,seeking multiple opinions can help you feel be...,Claim,Claim 1,124 125 126 127 128 129 130 131 132 133 134 13...
144286,4C471936CD75,1.618025e+12,861.0,954.0,getting multiple opinions will increase the pr...,Claim,Claim 2,146 147 148 149 150 151 152 153 154 155 156 15...
144287,4C471936CD75,1.618025e+12,1210.0,1967.0,if the President thinks that this is the best ...,Evidence,Evidence 1,204 205 206 207 208 209 210 211 212 213 214 21...


In [6]:
df["discourse_type_serial"]=df["discourse_type"].copy()
df["discourse_type_serial"]=df["discourse_type_serial"].apply(lambda x: encoding_discourse_type(x))
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,discourse_type_serial
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,0
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,1
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,2
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,2
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,3
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,2
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,2
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,1
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,2


In [7]:
corpus=[]

# now to operate on strings and text will use "re" module...
for i in range(144293):
    #basically it will replace all the characters which are not aplhabets(lower or upper),that is punctuations will be replaced
    review=re.sub('[^a-zA-Z]',' ',df["discourse_text"][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    # now we will remove irrevelant words isong stopwords and do stemming only on proper english words....
    all_stopwords=stopwords.words("english")
    all_stopwords.remove("not")
    review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
    review=' '.join(review)
    corpus.append(review)
    

In [8]:
corpus

['modern human today alway phone alway phone hour day stop text back forward group chat social media even drive',
 'realli bad consequ stuff happen come phone',
 'certain area unit state ban phone class room',
 'peopl phone know certain app app like facebook twitter instagram snapchat like friend move away want contact still contact post video text messag peopl alway differ way commun phone phone chang due gener',
 'drive one way get around peopl alway phone caus seriou problem',
 'thing call text drive realli import thing rememb peopl still think stupid matter still obey way save',
 'sometim news either accid suicid might involv someon not look go tweet someon sent either injuri death mysteri number say go kill know live know person contact make puzzl make start freak end realli badli',
 'phone fine use also best way come help',
 'go problem find help alway phone even though phone use almost everi day long safe would come use get troubl make sure not like phone middl drive',
 'news al

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=107)
# max_features's value is set at random.... u can change the value and test the result
X=cv.fit_transform(corpus).toarray()
y=df["discourse_type_serial"].values
print(X.shape)
print(y.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
from sklearn.svm import SVC
rbf_model = SVC(kernel='rbf')
y_svm=classifier.predict(X_test)
print(accuracy_score(y_test, y_svm))

(144293, 2700)
(144293,)
[[5 3]
 [5 3]
 [2 2]
 ...
 [5 3]
 [6 2]
 [0 0]]
[[ 340  692  147    3   80  376  257]
 [  33 2611    6    5   17  288  135]
 [ 936  976 2768   11  645 1967 1816]
 [ 187 5835   70   52  102 2770  918]
 [ 179  884  168    3  406  545  603]
 [  22  573    9    1    9  480  103]
 [  20  398   14    1    8  235  155]]
0.2360442149762639
0.2360442149762639
